In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from time import time
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mutual_info_score
from math import log2
from scipy.stats import entropy
from scipy import stats


from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from time import time
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report

import warnings
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [5]:
train = pd.read_csv("./Datasets/Customer_Segmentation_4(20A)/customer_segmentation_data.csv")
data = train.copy()
print(train.shape)
train.head()

(53503, 20)


,Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group
0,84966,23,Female,Married,Associate Degree,Mizoram,Entrepreneur,70541,policy5,04-10-2018,Phone,policy2,366603,2749,Group,Email,In-Person Meeting,Afternoon,English,Segment5
1,95568,26,Male,Widowed,Doctorate,Goa,Manager,54168,policy5,11-06-2018,Chat,policy1,780236,1966,Group,Mail,In-Person Meeting,Morning,French,Segment5
2,10544,29,Female,Single,Associate Degree,Rajasthan,Entrepreneur,73899,policy5,06-05-2021,Email,policy3,773926,4413,Group,Email,Mail,Evening,German,Segment3
3,77033,20,Male,Divorced,Bachelor's Degree,Sikkim,Entrepreneur,63381,policy5,09-02-2018,Chat,policy2,787815,4342,Family,Text,In-Person Meeting,Anytime,French,Segment3
4,88160,25,Female,Separated,Bachelor's Degree,West Bengal,Manager,38794,policy1,09-10-2018,Chat,policy4,366506,1276,Family,Email,Text,Weekends,English,Segment2


In [3]:
train = train.dropna()
t_data = train.copy()
label_encoders = {}
for col in t_data.columns:
    label_encoders[col] = LabelEncoder()
    t_data[col] = label_encoders[col].fit_transform(t_data[col])

t_data

NameError: name 't_data' is not defined

In [8]:
km = KModes(n_clusters=5, init='Huang', n_init=5, verbose=0)
clusters = km.fit_predict(t_data)

In [13]:
t_data['Cluster'] = clusters
train["Segmentation Group"] = clusters
train

,Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group
0,84966,23,Female,Married,Associate Degree,Mizoram,Entrepreneur,70541,policy5,04-10-2018,Phone,policy2,366603,2749,Group,Email,In-Person Meeting,Afternoon,English,0
1,95568,26,Male,Widowed,Doctorate,Goa,Manager,54168,policy5,11-06-2018,Chat,policy1,780236,1966,Group,Mail,In-Person Meeting,Morning,French,2
2,10544,29,Female,Single,Associate Degree,Rajasthan,Entrepreneur,73899,policy5,06-05-2021,Email,policy3,773926,4413,Group,Email,Mail,Evening,German,1
3,77033,20,Male,Divorced,Bachelor's Degree,Sikkim,Entrepreneur,63381,policy5,09-02-2018,Chat,policy2,787815,4342,Family,Text,In-Person Meeting,Anytime,French,2
4,88160,25,Female,Separated,Bachelor's Degree,West Bengal,Manager,38794,policy1,09-10-2018,Chat,policy4,366506,1276,Family,Email,Text,Weekends,English,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53498,44809,35,Female,Divorced,Associate Degree,Andaman and Nicobar Islands,Salesperson,120850,policy3,01-01-2019,Mobile App,policy1,586401,4404,Family,In-Person Meeting,Phone,Afternoon,German,1
53499,65485,61,Male,Single,Doctorate,Himachal Pradesh,Entrepreneur,122309,policy5,5/18/2021,Mobile App,policy4,637733,1285,Group,Text,Mail,Afternoon,German,4
53500,26213,25,Male,Divorced,Doctorate,Assam,Teacher,49258,policy2,11/27/2018,In-Person,policy1,631057,4407,Individual,Text,Text,Weekends,French,3
53501,63136,42,Male,Married,Doctorate,Andhra Pradesh,Artist,66301,policy4,06-04-2021,In-Person,policy1,730385,4482,Business,Mail,Phone,Morning,French,3


In [14]:
def calculate_gain_ratio(X, y, feature):

    # Calculate information gain
    info_gain = mutual_info_score(X[feature], y)
    
    # Calculate split information
    split_info = entropy(X[feature].value_counts(normalize=True), base=2)
    
    # Avoid division by zero
    if split_info == 0:
        return np.inf
    
    # Calculate gain ratio
    gain_ratio = info_gain / split_info
    
    return gain_ratio

def calculate_gini(y):

    # Count occurrences of each class
    class_counts = y.value_counts()
    
    # Calculate probabilities
    probabilities = class_counts / len(y)
    
    # Calculate Gini impurity
    gini_impurity = 1 - np.sum(probabilities ** 2)
    
    return gini_impurity

def calculate_gini_index(X, y, feature):

    # Get unique values of the feature
    values = X[feature].unique()
    
    gini_index = 0
    # Calculate weighted average of Gini impurities for each value of the feature
    for value in values:
        subset_y = y[X[feature] == value]
        subset_weight = len(subset_y) / len(y)
        gini_index += subset_weight * calculate_gini(subset_y)
    
    return gini_index


In [39]:
start = time()

# Separate features and target variable
X = train.drop(['Segmentation Group'], axis=1).copy()  # Features
y = train['Segmentation Group']  # Target variable

df_numerical = train.select_dtypes(include=['int', 'float']).copy()
df_numerical['Segmentation Group'] = train['Segmentation Group'].copy()

df_categorical = train.select_dtypes(include=['object']).copy()
target = 'Segmentation Group'
# Calculate information gain for each feature
NumX = X.select_dtypes(include=['int', 'float']).columns
info_gain = []
for col in X.columns:
    if col in NumX:
        num_bins = 3  # You can adjust this number according to your preference
        bin_width = (X[col].max() - X[col].min()) / num_bins
        
        # Create bins using equal-width binning
        bins = [X[col].min()-1 + i * bin_width for i in range(num_bins)]
        bins.append(X[col].max())
        
        # Binning numerical data
        X[col] = pd.cut(X[col], bins=bins, labels=[f'Bin_{i+1}' for i in range(num_bins)])
        info_gain.append(mutual_info_score(X[col], y))
    else:
        info_gain.append(mutual_info_score(X[col], y))
        # print('cat:',col)
        # info_gain.append(information_gain(train, col, target))




# Print information gain for each feature
feature_metrics = pd.DataFrame({"Features": X.columns})
feature_metrics['Info_Gain'] = info_gain

gain_ratios = {}
for col in X.columns:
    gain_ratios[col] = calculate_gain_ratio(X, y, col)

# Print gain ratio for each feature
# gain_ratios
feature_metrics['Gain_Ratio'] = gain_ratios.values()


# Calculate Gini impurity for the entire dataset
gini_impurity = calculate_gini(y)
print(f"Gini impurity for the entire dataset: {gini_impurity:.4f}")




# Gini Index....

gini_indices = {}
for feature in X.columns:
    gini_indices[feature] = calculate_gini_index(X, y, feature)


feature_metrics['Gini_Index'] = gini_indices.values()

# gini_impurities = {}
# for feature in X.columns:
#     feature_gini = calculate_gini_impurity(y[X[feature] == X[feature]])
#     gini_impurities[feature] = feature_gini


# feature_metrics['Gini'] = gini_impurities.values()


# print(feature_metrics)

print(f"Run Time: {time()-start:.3f} sec")
sorted_df = feature_metrics.sort_values(by='Gain_Ratio', ascending=False) # Gain Ratio is better for categorical data
print("Sorting According to: Gain Ratio")
sorted_df

Gini impurity for the entire dataset: 0.7746
Run Time: 8.603 sec
Sorting According to: Gain Ratio


,Features,Info_Gain,Gain_Ratio,Gini_Index
2,Gender,0.076574,0.076617,0.733291
14,Policy Type,0.078559,0.040213,0.735354
4,Education Level,0.089702,0.038806,0.732302
18,Preferred Language,0.083606,0.036727,0.737977
8,Behavioral Data,0.081405,0.035863,0.738319
11,Insurance Products Owned,0.080521,0.035182,0.737257
17,Preferred Contact Time,0.076322,0.033564,0.744596
3,Marital Status,0.067215,0.029269,0.746287
15,Customer Preferences,0.058224,0.025142,0.752863
10,Interactions with Customer Service,0.044965,0.019613,0.759132


In [40]:
# selected_df_cols = list(sorted_df.Features)
selected_df_cols = list(sorted_df.head(15).Features)
selected_df_cols.append("Segmentation Group")
selected_df = train[selected_df_cols].copy()
selected_df.head()

,Gender,Policy Type,Education Level,Preferred Language,Behavioral Data,Insurance Products Owned,Preferred Contact Time,Marital Status,Customer Preferences,Interactions with Customer Service,Occupation,Preferred Communication Channel,Purchase History,Geographic Information,Age,Segmentation Group
0,Female,Group,Associate Degree,English,policy5,policy2,Afternoon,Married,Email,Phone,Entrepreneur,In-Person Meeting,04-10-2018,Mizoram,23,0
1,Male,Group,Doctorate,French,policy5,policy1,Morning,Widowed,Mail,Chat,Manager,In-Person Meeting,11-06-2018,Goa,26,2
2,Female,Group,Associate Degree,German,policy5,policy3,Evening,Single,Email,Email,Entrepreneur,Mail,06-05-2021,Rajasthan,29,1
3,Male,Family,Bachelor's Degree,French,policy5,policy2,Anytime,Divorced,Text,Chat,Entrepreneur,In-Person Meeting,09-02-2018,Sikkim,20,2
4,Female,Family,Bachelor's Degree,English,policy1,policy4,Weekends,Separated,Email,Chat,Manager,Text,09-10-2018,West Bengal,25,0


In [41]:
label_encoders = {}
for col in selected_df.columns:
    label_encoders[col] = LabelEncoder()
    selected_df[col] = label_encoders[col].fit_transform(selected_df[col])

selected_df.head()

,Gender,Policy Type,Education Level,Preferred Language,Behavioral Data,Insurance Products Owned,Preferred Contact Time,Marital Status,Customer Preferences,Interactions with Customer Service,Occupation,Preferred Communication Channel,Purchase History,Geographic Information,Age,Segmentation Group
0,0,2,0,0,4,1,0,1,0,4,3,1,270,22,5,0
1,1,2,2,1,4,0,3,4,2,0,5,1,942,10,8,2
2,0,2,0,2,4,2,2,3,0,1,3,2,387,27,11,1
3,1,1,1,1,4,1,1,0,4,0,3,1,582,28,2,2
4,0,1,1,0,0,3,4,2,0,0,5,4,630,34,7,0


In [42]:
x_df= selected_df.drop("Segmentation Group", axis='columns').copy()
y_df= selected_df['Segmentation Group'].copy()
sp = train_test_split(x_df,y_df,test_size=0.33, random_state=14)
x_train = sp[0]
x_test = sp[1]
y_train = sp[2]
y_test = sp[3]

Naive Bayes

In [49]:
naiveBayes = GaussianNB()
naiveBayes.fit(x_train,y_train)
naiveBayes_y_pred = naiveBayes.predict(x_test)
nb_result = classification_report(y_test, naiveBayes_y_pred, output_dict=True)
print(f"Accuracy: {(nb_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(nb_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(nb_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(nb_result.get('weighted avg').get('f1-score')):.3f}")

Accuracy: 52.67%
Precision: 0.522
Recall: 0.527
F1-Score: 0.509


Decision Tree

In [45]:
decision_tree = DecisionTreeClassifier(max_leaf_nodes=200)
decision_tree.fit(x_train,y_train)
decision_tree_y_pred = decision_tree.predict(x_test)
dt_result = classification_report(y_test, decision_tree_y_pred, output_dict=True)
print(f"Accuracy: {(dt_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(dt_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(dt_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(dt_result.get('weighted avg').get('f1-score')):.3f}")
warnings.filterwarnings('ignore')

Accuracy: 57.24%
Precision: 0.562
Recall: 0.572
F1-Score: 0.562


Random Forest

In [46]:
random_forest = RandomForestClassifier()
random_forest.fit(x_train,y_train)
random_forest_y_pred = random_forest.predict(x_test)
rf_result = classification_report(y_test, random_forest_y_pred, output_dict=True)
print(f"Accuracy: {(rf_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(rf_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(rf_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(rf_result.get('weighted avg').get('f1-score')):.3f}")

Accuracy: 72.93%
Precision: 0.730
Recall: 0.729
F1-Score: 0.721


Bagging

In [47]:
start = time()
bagging = BaggingClassifier(n_estimators=200)
bagging.fit(x_train, y_train)
bagging_y_pred = bagging.predict(x_test)
bagging_result = classification_report(y_test, bagging_y_pred, output_dict=True)
print(f"Accuracy: {(bagging_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(bagging_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(bagging_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(bagging_result.get('weighted avg').get('f1-score')):.3f}")
print(f"Run Time: {time()-start:.3f}")

Accuracy: 71.32%
Precision: 0.709
Recall: 0.713
F1-Score: 0.708
Run Time: 31.098


AdaBoost

In [48]:
start = time()
adaboost = AdaBoostClassifier(n_estimators=300, random_state=1)
adaboost.fit(x_train, y_train)
adaboost_y_pred = adaboost.predict(x_test)
adaboost_result = classification_report(y_test, adaboost_y_pred, output_dict=True)
print(f"Accuracy: {(adaboost_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(adaboost_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(adaboost_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(adaboost_result.get('weighted avg').get('f1-score')):.3f}")
print(f"Run Time: {time()-start:.3f}")
warnings.filterwarnings('ignore')

Accuracy: 94.68%
Precision: 0.947
Recall: 0.947
F1-Score: 0.947
Run Time: 9.395
